# OpenAI API 사전 준비

In [1]:
# 패키지 설치
!pip install -U openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 2.12.0
    Uninstalling openai-2.12.0:
      Successfully uninstalled openai-2.12.0


In [2]:
import os
from google.colab import userdata

# 환경 변수 준비 (왼쪽 끝 키 아이콘으로 OPENAI_API_KEY 설정)
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [3]:
from openai import OpenAI

# 클라이언트 준비
client = OpenAI()

# 이미지 생성

In [4]:
# 추론 실행
response = client.responses.create(
    model="gpt-5",
    input="귀여운 고양이 귀 메이드 그림을 만들어줘",
    tools=[{"type": "image_generation"}],
)

In [5]:
import base64

# 이미지 파일 저장
image_data = [
    output.result
    for output in response.output
    if output.type == "image_generation_call"
]
if image_data:
    image_base64 = image_data[0]
    with open("output.png", "wb") as f:
        f.write(base64.b64decode(image_base64))

# 멀티턴 이미지 생성

In [6]:
# 2턴째 추론 실행
response_fwup = client.responses.create(
    model="gpt-5",
    previous_response_id=response.id, # 응답 ID
    input="카페에서 일하는 모습을 그려줘. 가로가 긴 이미지로.",
    tools=[{"type": "image_generation"}],
)

In [7]:
# 이미지 파일 저장
image_data_fwup = [
    output.result
    for output in response_fwup.output
    if output.type == "image_generation_call"
]
if image_data_fwup:
    image_base64 = image_data_fwup[0]
    with open("output2.png", "wb") as f:
        f.write(base64.b64decode(image_base64))

# 수정 프롬프트

In [8]:
# 수정 프롬프트 확인
for item in response.output:
    # image_generation_call
    if item.type == "image_generation_call":
        print("==image_generation_call==")
        print(item.revised_prompt)

==image_generation_call==
밝고 사랑스러운 분위기의 일러스트. 20대 성인 여성 캐릭터, 고양이 귀 헤어밴드를 한 메이드. 전통적인 흑백 메이드복(어깨 퍼프, 긴 소매, 무릎 길이 스커트, 앞치마)으로 노출 없이 단정하고 깔끔한 디자인. 귀여운 표정으로 미소 지으며, 한 손으로 가볍게 인사하거나 작은 쟁반에 찻잔을 들고 있는 포즈. 큰 반짝이는 눈, 도톰한 볼, 부드러운 파스텔 톤, 깨끗한 라인아트. 카페 내부 배경은 따스한 오전 햇살이 들어오고, 살짝 흐림(얕은 피사계 심도)으로 캐릭터가 돋보이게. 고양이 귀는 자연스러운 모피 질감, 작게 보이는 꼬리. 전체적으로 따뜻한 색감, 고화질(4K), 깔끔한 셀셰이딩과 부드러운 그라데이션 조화. 캐릭터 비율은 성인 체형(8등신에 가까움). 세로형(3:4) 구성.


# 스트리밍

In [9]:
# 스트리밍으로 추론 실행
stream = client.responses.create(
    model="gpt-5",
    input="고양이가 햇볕을 쬐고 있는 일러스트를 그려줘",
    stream=True,
    tools=[{"type": "image_generation", "partial_images": 2}],
)

In [10]:
import base64

# 이미지 파일 저장
for event in stream:
    if event.type == "response.image_generation_call.partial_image":
        idx = event.partial_image_index
        image_base64 = event.partial_image_b64
        image_bytes = base64.b64decode(image_base64)
        with open(f"streaming{idx}.png", "wb") as f:
            f.write(image_bytes)